In [1]:
import tensorflow.keras
from PIL import Image, ImageOps

import numpy as np
import cv2

#from motor import *
def mark_img(img, blue_threshold=200, green_threshold=200, red_threshold=200): # 1. 흰색 차선 찾기
    mark = img
    #  BGR 제한 값 (기준)
    bgr_threshold = [blue_threshold, green_threshold, red_threshold]

    # BGR 제한 값(기준)보다 작으면 검은색으로
    thresholds = (img[:,:,0] < bgr_threshold[0]) \
                | (img[:,:,1] < bgr_threshold[1]) \
                | (img[:,:,2] < bgr_threshold[2])
    mark[thresholds] = [0,0,0] # 검정색으로 
    return mark
 
def grayscale(img): # 흑백이미지로 변환
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def region_of_interest(img, vertices, color3=(255,255,255), color1=255): # ROI 셋팅
    mask = np.zeros_like(img) # mask = img와 같은 크기의 빈 이미지
    
    if len(img.shape) > 2: # Color 이미지(3채널)라면 :
        color = color3
    else: # 흑백 이미지(1채널)라면 :
        color = color1
        
    # vertices에 정한 점들로 이뤄진 다각형부분(ROI 설정부분)을 color로 채움 
    cv2.fillPoly(mask, vertices, color)
    # 이미지와 color로 채워진 ROI를 합침
    ROI_image = cv2.bitwise_and(img, mask)

    return ROI_image

# 허프변환 라인 그리기 (확인용)
def draw_lines(img, lines, color=[255, 255, 0], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)\
            
# 허프 변환
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap): 
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)

    return lines

# 두 이미지 operlap 하기
def weighted_img(img, initial_img, α=1, β=1., λ=0.): 
    return cv2.addWeighted(initial_img, α, img, β, λ)

# '대표선' 구하기 (62, 63줄 코드 이해 X )
def get_fitline(img, f_lines): 
    lines = np.squeeze(f_lines)
    if(len(lines.shape) == 1): #선이 하나밖에 없는 경우 배열의 모양 따로 조정
        lines = lines.reshape(2,2)
    else:
        lines = lines.reshape(lines.shape[0]*2,2)

    rows,cols = img.shape[:2]
    output = cv2.fitLine(lines,cv2.DIST_L2,0, 0.01, 0.01)
    vx, vy, x, y = output[0], output[1], output[2], output[3]
    x1, y1 = int(((img.shape[0]/2+70)-y)/vy*vx + x) , int(img.shape[0]/2+70)
    x2, y2 = int(((img.shape[0]/2-25)-y)/vy*vx + x) , int(img.shape[0]/2-25)
    result = [x1,y1,x2,y2]
    
    return result

# '대표선' 그리기
def draw_fit_line(img, lines, color=[255, 0, 0], thickness=10):
        # cv2.line(이미지, 시작좌표(0, 0), 끝좌표(500, 500), 색깔, 두께)
        cv2.line(img, (lines[0], lines[1]), (lines[2], lines[3]), color, thickness) 

# '대표선' 이용해서 -> 소실점 구하기 (공식)
def expression(x1,y1,x2,y2,x3,y3,x4,y4):
    m_a = (y2 - y1) / (x2 -x1)
    m_b = (y4 - y3) / (x4 - x3)
    n_a = -((y2 - y1) / (x2 - x1) * x1 ) + y1
    n_b = -((y4 - y3) / (x4 -x3) * x3 ) + y3
    x = (n_b - n_a) / (m_a - m_b) 
    y = m_a * ((n_b - n_a) / (m_a - m_b)) + n_a 

    return x,y
 

#=======================================

 

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)


# Load the model
model = tensorflow.keras.models.load_model('keras_model.h5',compile = False)

 
# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)


cap = cv2.VideoCapture(0)



while(cap.isOpened()):

    ret,image = cap.read()

    image2 = cv2.resize(image, (224, 224)) # window 사이즈 조정
    

    height, width = image2.shape[:2] # 이미지 높이, 너비
    
    #cv2.imshow('result2',image2)
    #cv2.imshow('result2',image2) # 결과 이미지 출력
 
    
    #turn the image into a numpy array

    image_array = np.asarray(image2)

    

    # Normalize the image

    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
    
 

    # Load the image into the array

    data[0] = normalized_image_array

    # run the inference

    prediction = model.predict(data)

    m_width = int(width/2)

    mrk_img = mark_img(image2)
    gray_img = grayscale(mrk_img)
    ####################################################################
    vertices = np.array([[(0,0),
                          (width,0),
                          (width,height),
                          (0,height)]], dtype=np.int32)
    ####################################################################
    ROI_img = region_of_interest(gray_img, vertices) # ROI 설정
    ####################################################################
    rho = 1
    theta = 1 * np.pi/180
    threshold = 30    # threshold 값이  작으면 그만큼 기준이 낮아져 많은 직선이 검출될 것이고, 값을 높게 정하면 그만큼 적지만 확실한 직선들만 검출이 될 것이다
    ####################################################################
    
    line_arr = hough_lines(ROI_img, rho, theta, threshold, 10, 20) # 허프 변환
    line_arr = np.squeeze(line_arr) # remove single dimension (차원을 하나 줄임)

    # 기울기 구하기 (arctan(y,x)이용)
    slope_degree = np.arctan2(line_arr[:,1] - line_arr[:,3], line_arr[:,0] - line_arr[:,2]) * 180 / np.pi

# 수평 기울기 제한
    line_arr = line_arr[np.abs(slope_degree)<175]
    slope_degree = slope_degree[np.abs(slope_degree)<175]
# 수직 기울기 제한
    line_arr = line_arr[np.abs(slope_degree)>95]
    slope_degree = slope_degree[np.abs(slope_degree)>95]
# 필터링된 직선 버리기
    L_lines, R_lines = line_arr[(slope_degree>0),:], line_arr[(slope_degree<0),:]
    L_lines, R_lines = L_lines[:,None], R_lines[:,None]

    if(len(L_lines) == 0 and len(R_lines) == 0): #L_lines, R_lines 모두 없는 경우
        L_lines = pre_left_line
        R_lines = pre_right_line
    elif(len(L_lines) == 0):#L_lines만 없는 경우
        L_lines = pre_left_line
        pre_right_line = R_lines
    elif(len(R_lines) == 0):#R_lines만 없는 경우
        R_lines = pre_right_line
        pre_left_line = L_lines
    else:#라인 모두 검출한 경우
        pre_right_line = R_lines
        pre_left_line = L_lines

    temp = np.zeros((image2.shape[0], image2.shape[1], 3), dtype=np.uint8)

# 왼쪽, 오른쪽 각각 대표선 구하기
    left_fit_line = get_fitline(image2,L_lines)
    right_fit_line = get_fitline(image2,R_lines)
    #print(left_fit_line) # [158, 539, 388, 370] -> 단 1개 검출 

# 대표선 '그리기'
    draw_fit_line(temp, left_fit_line)
    draw_fit_line(temp, right_fit_line)
    #print('left_fit_line = ', left_fit_line) # [158, 539, 388, 370]

    vanishing_point = expression(left_fit_line[0],left_fit_line[1],left_fit_line[2],left_fit_line[3],right_fit_line[0],right_fit_line[1],right_fit_line[2],right_fit_line[3])
    #print(vanishing_point) # (476.9880952380953, 304.61309523809524)...

    v_x = int(vanishing_point[0])
    v_y = int(vanishing_point[1])

    result = weighted_img(temp, image2) # 원본 이미지(=image)에 검출된 선(=temp) overlap
    cv2.circle(result, (v_x,v_y), 6, (0,0,255), -1) # cv2.circle(image, center_coordinates, radius, color, thickness)

    #circle 기준선(보조선)
    cv2.line(result,(m_width,0),(m_width,300),(255,255,0),5) # cv2.line(image, start_point, end_point, color, thickness)
    
    if(m_width != v_x):
        slope_degree = float(v_y) / (m_width - v_x)
    else:
        slope_degree = 0
    print(slope_degree)
    if(slope_degree > 2.475): # degree 22
        print("Right!!!")
        #cv2.circle(result,(1000,50), 6,(0,0,255),-1) # (1000,50)에 circle 찍어라 
    elif(slope_degree < -2.475): # 소실점의 x좌표가 중앙선보다 왼쪽에 있을때
        print("Left!!!")
        #cv2.circle(result,(100,50), 6,(0,0,255),-1) # (100,50)에 circle 찍어라
    else:
        print("foward!!!")
        
    
    if prediction[0][0] >= 0.8:
        #direction = F
        print('forward!!')
    elif prediction[0][1] >= 0.8:
        #direction = FR
        print('right!!')
    
    cv2.imshow('TM',result) # 결과 이미지 출력
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
-1.0389610389610389
foward!!!
forward!!
-2.8125
Left!!!
-2.6470588235294117
Left!!!
-1.7254901960784315
foward!!!
forward!!
-2.292682926829268
foward!!!
-2.473684210526316
foward!!!
forward!!
-2.4
foward!!!
forward!!
-2.4864864864864864
Left!!!
forward!!
-1.6071428571428572
foward!!!
forward!!
-4.222222222222222
Left!!!
-3.8947368421052633
Left!!!
-3.619047619047619
Left!!!
forward!!
-3.4347826086956523
Left!!!
forward!!
-3.0384615384615383
Left!!!
forward!!
-3.3333333333333335
L

-3.0625
Left!!!
-3.357142857142857
Left!!!
-3.2413793103448274
Left!!!
-3.4444444444444446
Left!!!
-3.2758620689655173
Left!!!
-3.8333333333333335
Left!!!
-3.7916666666666665
Left!!!
-4.333333333333333
Left!!!
-2.9696969696969697
Left!!!
-3.68
Left!!!
-2.9696969696969697
Left!!!
-2.8823529411764706
Left!!!
-3.2413793103448274
Left!!!
-3.6538461538461537
Left!!!
-3.2413793103448274
Left!!!
-3.64
Left!!!
-2.7777777777777777
Left!!!
-3.357142857142857
Left!!!
-4.7368421052631575
Left!!!
-3.310344827586207
Left!!!
-3.4074074074074074
Left!!!
-3.3703703703703702
Left!!!
-3.5384615384615383
Left!!!
-3.064516129032258
Left!!!
-3.2758620689655173
Left!!!
-3.68
Left!!!
-4.136363636363637
Left!!!
-3.68
Left!!!
-3.5384615384615383
Left!!!
-3.9565217391304346
Left!!!
-4.333333333333333
Left!!!
-3.129032258064516
Left!!!
-3.357142857142857
Left!!!
-3.7083333333333335
Left!!!
-3.129032258064516
Left!!!
-3.4814814814814814
Left!!!
-3.096774193548387
Left!!!
-3.064516129032258
Left!!!
-3.75
Left!!!
-3

-2.7142857142857144
Left!!!
forward!!
-2.7941176470588234
Left!!!
forward!!
-2.764705882352941
Left!!!
forward!!
-2.6216216216216215
Left!!!
-3.3333333333333335
Left!!!
-3.3846153846153846
Left!!!
-3.2962962962962963
Left!!!
-2.823529411764706
Left!!!
-2.7714285714285714
Left!!!
-3.0
Left!!!
-3.1724137931034484
Left!!!
forward!!
-2.823529411764706
Left!!!
-3.1379310344827585
Left!!!
-3.032258064516129
Left!!!
forward!!
-2.764705882352941
Left!!!
-3.1
Left!!!
-2.967741935483871
Left!!!
-2.7714285714285714
Left!!!
-2.5128205128205128
Left!!!
forward!!
-3.0
Left!!!
-3.103448275862069
Left!!!
-3.625
Left!!!
-2.90625
Left!!!
forward!!
-2.6470588235294117
Left!!!
-3.1785714285714284
Left!!!
forward!!
-2.9375
Left!!!
forward!!
-3.4
Left!!!
forward!!
-3.6
Left!!!
forward!!
-2.4634146341463414
foward!!!
forward!!
-3.5833333333333335
Left!!!
forward!!
-3.0
Left!!!
forward!!
-1.3384615384615384
foward!!!
forward!!
-2.5128205128205128
Left!!!
forward!!
-2.9375
Left!!!
-3.103448275862069
Left!!!
fo